### What Needs To Be Done ###



### Train Model ###

In [195]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense
from keras.callbacks import EarlyStopping

# Read the source text from a file
file_path = 'source_text.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    source_text = file.read()

# Custom tokenizer that does not filter out punctuation (except quotes and double quotes)
tokenizer = Tokenizer(filters='"#$%&()*+-/:;<=>@[\\]^_{|}~\t\n')
tokenizer.fit_on_texts([source_text])
sequence = tokenizer.texts_to_sequences([source_text])[0]
total_words = len(tokenizer.word_index) + 1

# Generate input sequences for training
input_sequences = [sequence[:i] for i in range(1, len(sequence))]

# Calculate average sequence length before padding and adjust it if needed
average_sequence_len = np.mean([len(seq) for seq in input_sequences])
max_sequence_len = int(average_sequence_len * 1.5)
print(f"Average sequence length: {average_sequence_len}, Max sequence length after padding: {max_sequence_len}")

# Pad sequences to the same length
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
target_word = to_categorical(sequence[1:], num_classes=total_words)

# Adjust the model parameters
min_threshold = 10
embedding_dim = max(min_threshold, min(50, total_words // 20))
gru_units = max(min_threshold, min(50, total_words // 40))
print(f"Embedding dimensions: {embedding_dim}, GRU units: {gru_units}")
# Model definition
model = Sequential()
model.add(Embedding(total_words, embedding_dim, input_length=max_sequence_len))
model.add(GRU(units=gru_units, return_sequences=False))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Define the batch size
batch_size = max(32, min(1024, total_words // 10))
print(f"Batch size: {batch_size}")

# Define Early Stopping callback
early_stopping = EarlyStopping(monitor='loss', patience=5)

# Train the model
model.fit(input_sequences, target_word, epochs=100, verbose=1, callbacks=[early_stopping], batch_size=batch_size)


Average sequence length: 1071.5, Max sequence length after padding: 1607
Embedding dimensions: 50, GRU units: 25
Batch size: 103
Epoch 1/100
21/21 [==============================] - 17s 660ms/step - loss: 6.9328
Epoch 2/100
21/21 [==============================] - 14s 651ms/step - loss: 6.8941
Epoch 3/100
21/21 [==============================] - 14s 681ms/step - loss: 6.6239
Epoch 4/100
21/21 [==============================] - 15s 693ms/step - loss: 6.1959
Epoch 5/100
21/21 [==============================] - 15s 696ms/step - loss: 6.0575
Epoch 6/100
21/21 [==============================] - 15s 736ms/step - loss: 6.0223
Epoch 7/100
21/21 [==============================] - 15s 717ms/step - loss: 6.0139
Epoch 8/100
21/21 [==============================] - 14s 670ms/step - loss: 6.0106
Epoch 9/100
21/21 [==============================] - 14s 670ms/step - loss: 6.0068
Epoch 10/100
21/21 [==============================] - 14s 667ms/step - loss: 5.9991
Epoch 11/100
21/21 [====================

### Save Model ###
This should be ran every time a model is trained and is tested

In [199]:
from datetime import datetime
import os

base_dir = 'models'
os.makedirs(base_dir, exist_ok=True)
# After training, create a timestamp or a unique identifier for the model
model_id = datetime.now().strftime("%Y%m%d-%H%M%S")
model_name = f"model_{model_id}.h5"
model_path = os.path.join(base_dir, model_name)

# Save the model to the specified directory
model.save(model_path)
print(f"Model saved to {model_path}")

# Later, to load the model, you can use:
# model = load_model(model_path)

Model saved to models\model_20231104-165145.h5


## Generate Next Word ###
This just generates the next immediate word. Number of words generated can be adjusted

In [241]:
import numpy as np

def sample(preds, temperature=1.0):
    # Convert predictions to probabilities
    preds = np.asarray(preds).astype('float64')
    
    # Apply temperature scaling
    preds = np.log(preds + 1e-7) / temperature
    exp_preds = np.exp(preds)
    
    # Normalize predictions
    preds = exp_preds / np.sum(exp_preds)
    
    # Sample a single prediction with the probabilities to return a likely next word index
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text_seq(model, tokenizer, text, num_words, max_sequence_len, temperature=1.0):
    input_text = text
    for _ in range(num_words):
        # Convert the input text to a sequence of word indexes
        input_seq = tokenizer.texts_to_sequences([input_text])[0]

        # Pad the sequence to the required length
        input_seq = pad_sequences([input_seq], maxlen=max_sequence_len, padding='pre')

        # Predict the next word index
        predictions = model.predict(input_seq, verbose=0)[0]
        predicted_word_index = sample(predictions, temperature)

        # Convert the index to a word
        predicted_word = tokenizer.index_word.get(predicted_word_index, '')

        # Append the predicted word to the input text
        input_text += ' ' + predicted_word

    return input_text.strip()

# Test the model on a new input sequence with temperature
test_text = "Hello, how are"
num_words = 1
temperature = 0.75  # Adjust the temperature as needed to vary randomness
generated_text = generate_text_seq(model, tokenizer, test_text, num_words, max_sequence_len, temperature)
print(generated_text)


Hello, how are the


### Generate Rest of Sentence ###
Generates words until a punctuation is found. There is a max word limit to prevent a feedback loop. 

If words a getting in a feedback loop or you are not happy with results, try adjusting the temperature before training again. Higher values gives more randomness while lower values has a higher likely hood to have a feedback loop

In [237]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-7) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text_seq(model, tokenizer, text, max_sequence_len, temperature=1.0, punctuations=".!?"):
    input_text = text
    word_count = 0
    max_words = 50
    print(input_text, end=' ')
    while True:
        word_count += 1
        if word_count > max_words:
            print("\nError: Max Words Reached Before Punctuation")
            break
        # Convert the input text to a sequence of word indexes
        input_seq = tokenizer.texts_to_sequences([input_text])[0]

        # Pad the sequence to the required length
        input_seq = pad_sequences([input_seq], maxlen=max_sequence_len, padding='pre')

        # Predict the next word index
        predictions = model.predict(input_seq, verbose=0)[0]
        predicted_word_index = sample(predictions, temperature)

        # Convert the index to a word
        predicted_word = tokenizer.index_word.get(predicted_word_index, '')

        # Append the predicted word to the input text and print it
        input_text += ' ' + predicted_word
        print(predicted_word, end=' ', flush=True)

        # Break if the predicted word ends with a punctuation mark or is empty
        if any(predicted_word.endswith(punct) for punct in punctuations) or predicted_word == '':
            break

    print()  # To ensure we move to a new line after the sentence ends
    return input_text.strip()

# Define a list of temperatures. For example, low=0.2, medium=0.7, high=1.2
temperatures = [0.1 , 0.2, 0.5 , 0.7, 0.8 , 0.85 , 0.9, 1.2, 2]

# Test the model on a new input sequence with different temperatures
test_text = "But remember, it's not just about winning; it's about the"
print("The input text:", test_text)
for temp in temperatures:
    print(f"\nGenerating with temperature {temp}:")
    generated_text = generate_text_seq(model, tokenizer, test_text, max_sequence_len, temp)


The input text: But remember, it's not just about winning; it's about the

Generating with temperature 0.1:
But remember, it's not just about winning; it's about the new within as dusk falls. 

Generating with temperature 0.2:
But remember, it's not just about winning; it's about the new language how are the force of the silent ambassador of his blade. 

Generating with temperature 0.5:
But remember, it's not just about winning; it's about the seedlings with a members. 

Generating with temperature 0.7:
But remember, it's not just about winning; it's about the opportunity to the best medicine, my secret is the cop just two guys, facades, saving the floor, a puppies doing with a little worlds. 

Generating with temperature 0.8:
But remember, it's not just about winning; it's about the mouths of graves, than the force of meant capable of an labor challenges on confirm lost guys, hues club. 

Generating with temperature 0.85:
But remember, it's not just about winning; it's about the seaso

### Notes ###
From Aaron, Not Chat GPT Generated Lol:

I believe the training takes the entire training set as one big context instead of training on each sentence as a single context. This is probably why it generates long senteces that don't make much sense. We can change how the training is tokenized to fix this but I don't think its a big deal for the scope of this project. We can just add this thought to the conclusion. The project says to use the embedddings from the Glove algorithm, this is not implemented yet nor do I know if Artsi actually wants us to do that.